In [1]:
import scrapy
import json
import logging
import pandas as pd
import numpy as np
from scrapy.crawler import CrawlerProcess

In [2]:
url = 'https://www.perfumeria.pl/collections/perfumy' # link do kategorii

In [3]:
# format pliku wynikowego
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('output.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
class ProductsSpider(scrapy.Spider):
    
    name = 'Products'
    start_urls = [url] 
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'output.json'
        ,'DOWNLOAD_DELAY': 3 # opóźnienie w sekundach
        ,'RANDOMIZE_DOWNLOAD_DELAY' : True # losowość opóźnień: (0.5,1.5)*lag
    }
    
    def parse_product(self,response):
        
        # scrapowanie danego produktu
        xpath_name = '//div[@class="col-info"]/h1/text()' # ścieżka do nazwy 
        xpath_EAN = "//script[@type='application/json' and text()[contains(.,'barcode')]]/text()" # ścieżka do kodu EAN
        xpath_cena =  '//p[@id = "ProductPrice"]/text()' # ścieżka do ceny promocyjnej
        xpath_regural =  '//span[@class = "price__sale"]/text()' # ścieżka do ceny regularnej
        
        name = response.xpath(xpath_name).get()
        EAN = str(json.loads(response.xpath(xpath_EAN).get())['variants'][0]['barcode'])
        cena = response.xpath(xpath_cena).get()
        regural = response.xpath(xpath_regural).get()
        
        yield {
                'Name': name,
                'EAN': EAN,
                'cena': regural,
                'cena promo': cena,
                'url': response.url
                }
        
        # scrapowanie poszczególnych wariantów produktów. Dany produkt może mieć kilka wariantów objętościowych
        xpath_option = '//div[@class = "option" and @data-value]//@href' # ścieżka do poszczególnych wariantów

        links = list(set(response.xpath(xpath_option).extract())) # tworzę unikalną listę wariantów (w kodzie HTML linki powtarzają się)
        links = ['https://www.perfumeria.pl' + x for x in links] # lista linków do kolejnych wariantów
        
        yield from response.follow_all(links, callback=self.parse_product) # kontynuuj scrapowanie dla każdego z nich

    def parse(self,response):
        
        xpath_url = '//a[@class = "product-img"]/@href' # link do produktu na karcie
        xpath_next_page = '//li[@class = "page__next"]/a/@href' # link do następnej karty
        
        selection = response.xpath(xpath_url) # produkty na danej zakladce
        
        for s in selection: # scrapowanie produktow na karcie
            href = 'https://www.perfumeria.pl' + s.get()
            yield scrapy.Request(href, self.parse_product) 
            
        next_page = response.xpath(xpath_next_page).get() # przechodzenie do nastepnej karty. Na potrzebe prezentacji wyłączone, żeby oszczędzić czas
        if next_page:
            yield response.follow('https://www.perfumeria.pl' + next_page, callback=self.parse)

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.34 Safari/537.36'
})

process.crawl(ProductsSpider)
process.start()

2023-02-03 01:52:47 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2023-02-03 01:52:47 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Windows-10-10.0.19041-SP0
2023-02-03 01:52:47 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/101.0.4951.34 Safari/537.36'}
2023-02-03 01:52:47 [py.warnings] WARNING: c:\users\norbert\appdata\local\programs\python\python39\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have n

In [6]:
dfjl = pd.read_json('output.jl', lines=True, dtype = str)
# usunięcie pustych znaków
dfjl['cena'] = dfjl['cena'].apply(lambda x: x.strip() if type(x) == str else x)
# # przesunięcie cen w lewo. Na stronie cena promo jest jako zwykła 
dfjl["cena"] = np.where(dfjl['cena'] == 'None', dfjl["cena promo"], dfjl["cena"])
dfjl["cena promo"] =np.where(dfjl["cena promo"] == dfjl["cena"], '', dfjl['cena promo'])
dfjl = dfjl[dfjl['cena promo'] != 'None']
# # usunięcie ' zł' z ceny i zamiana na liczbe
dfjl['cena promo'] = dfjl['cena promo'].apply(lambda x: x.replace(' zł','').replace(',','.') if type(x) == str else x)
dfjl['cena'] = dfjl['cena'].apply(lambda x: x.replace(' zł','').replace(',','.') if type(x) == str else x)
# zostawiam tylko promocje
dfjl_ = dfjl[dfjl['cena promo'] != ''] # możliwe, że na stronie nie będzie żadnych promocji i zostanie pusty df
# zamiana typu danych
dfjl_ = dfjl_.astype({'cena promo':'float','cena':'float'})

In [7]:
# zapisanie do excela
dfjl_.to_excel('Wynik2.xlsx', sheet_name='Arkusz', index = False, freeze_panes = [1,0])